In [1]:
from pytz import timezone
import matplotlib.pyplot as plt
import pandas as pd
import time
import datetime as dt
from datetime import datetime
import yfinance as yf
import pandas as pd
import pytz
import plotly.graph_objects as go
import pandas_datareader as pdr
import MetaTrader5 as mt5

utc_tz = timezone('UTC')
pd.set_option('display.max_columns', 500) # number of columns to be displayed
pd.set_option('display.width', 1500)      # max table width to display

In [2]:
MS = 'Bearish'
Symbol = 'EURCAD'

In [3]:
#Access MT5 package and Initialize the system (Generalized to be inluded in all scripts)
if not mt5.initialize():
        print("initialize() failed, error code =",mt5.last_error())
        quit()
mt5.login()
#Set parameters for time
timezone = pytz.timezone("UTC")
now = datetime.now(timezone)
start = datetime.now(timezone) - dt.timedelta(days=1)
utc_from = datetime(start.year, start.month, start.day)
utc_to = datetime(now.year, now.month, now.day, now.hour, now.minute, now.second)
#Pull price data from MT5
rates = mt5.copy_rates_range(Symbol, mt5.TIMEFRAME_M15, utc_from, utc_to)
#Format Data in Pandas DataFrames to be manipulated
htf = pd.DataFrame(rates)
htf['time']=pd.to_datetime(htf['time'], unit='s')

In [4]:
#htf.tail()

In [20]:
for i in range(2,htf.shape[0]):
    current = htf.iloc[i,:]
    prev = htf.iloc[i-1,:]
    prev_2 = htf.iloc[i-2,:]
    realbody = abs(current['open'] - current['close'])
    candle_range = current['high'] - current['low']
    idx = htf.index[i]
    htf.loc[idx,'Bullish swing'] = current['low'] > prev['low'] and prev['low'] < prev_2['low']
    htf.loc[idx,'Bearish swing'] = current['high'] < prev['high'] and prev['high'] > prev_2['high']
    #Finding the Bullish Market Shift
    if htf.loc[idx,'Bullish swing'] == True:
        htf.loc[idx,'Bull1'] = htf['low'].loc[htf.index[i-1]]
        Bull1 = htf.loc[htf.index[i-1]]
    if htf.loc[idx,'Bearish swing'] == True and Bull1['time'] < htf.loc[idx,'time']:
        htf.loc[idx,'Bull2'] = htf['high'].loc[htf.index[i-1]]
        Bull2 = htf.loc[htf.index[i-1]]
    if htf.loc[idx,'Bullish swing'] == True and Bull1['time'] < htf.loc[idx,'time']:
        htf.loc[idx,'Bull3'] = htf['low'].loc[htf.index[i-1]]
        Bull3 = htf.loc[htf.index[i-1]]
    #Finding the Bearish Market Shift
    if htf.loc[idx,'Bearish swing'] == True:
        htf.loc[idx,'Bear1'] = htf['high'].loc[htf.index[i-1]]
        Bear1 = htf.loc[htf.index[i-1]]
    if htf.loc[idx,'Bearish swing'] == True and Bear1['time'] < htf.loc[idx,'time']:
        htf.loc[idx,'Bear2'] = htf['low'].loc[htf.index[i-1]]
        Bear2 = htf.loc[htf.index[i-1]]
    if htf.loc[idx,'Bearish swing'] == True and Bear1['time'] < htf.loc[idx,'time']:
        htf.loc[idx,'Bear3'] = htf['high'].loc[htf.index[i-1]]
        Bear3 = htf.loc[htf.index[i-1]]

In [37]:
q = htf['Bear2'].tail(15).dropna()
f = q.iloc[-2:]
round(f.iloc[-2],5)

1.30399

In [22]:
if MS == 'Bullish':
    if htf.loc[idx,'Bullish swing'] == True:
        bull_low1 = Bull1['low']
    if Bull1['time'] < Bull2['time']:
        bull_high = Bull2['high']
        bulldis_high = bull_high
    if Bull2['time'] < Bull3['time']:
        bull_low2 = Bull3['low']
        bulldis_low = bull_low2
    if Bull1['low'] > Bull3['low']:
        start_time = Bull1['time']
        end_time = Bull3['time']
        print ('Start time is:',start_time)
        print ('End time is:',end_time)
        print ('Price Entry Zone is from', bulldis_low, 'to', bulldis_high)
    else:
        print ("No Pattern Found!")
else:
    if htf.loc[idx,'Bearish swing'] == True:
        Bear_low1 = Bear1['high']
    if Bear1['time'] < Bear2['time']:
        Bear_high = Bear2['low']
        beardis_high = Bear_high
    if Bear2['time'] < Bear3['time']:
        Bear_low2 = Bear3['high']
        beardis_low = Bear_low2
    if Bear1['low'] > Bear3['low']:
        start_time = Bear1['high']
        end_time = Bear3['time']
        print ('Start time is:',start_time)
        print ('End time is:',end_time)
        print ('Price Entry Zone is from', beardis_low, 'to', beardis_high)
    else:
        print ("No Pattern Found!")

No Pattern Found!


In [8]:
Bear_low1

1.30515

In [9]:
Bear_high

NameError: name 'Bear_high' is not defined